In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy
import pandas as pd

import theano
import theano.tensor as T

In [5]:
data = pd.read_pickle('../hws/HW1/prostate.df')
train_set_y = (data.values[:, -1] == 'tumor')
train_set_x = data.values[:, :-1]
n_genes = train_set_x.shape[1]

train_set_x = train_set_x[:100].astype(float)
train_set_y = train_set_y[:100].astype(float).reshape((100,1))
batch_size = 20
n_batches = len(train_set_y)/batch_size

print len(train_set_y), n_batches

train_set_x = theano.shared(value=train_set_x, name='tx', borrow=True)
train_set_y = theano.shared(value=train_set_y, name='ty', borrow=True)

100 5


In [6]:
from logreg import LogisticRegression

In [7]:
class HiddenLayer(object):
    
    def __init__(self, input, n_in, n_out):
        self.input = input
        
        W_values = 4*numpy.random.uniform(
                low=-numpy.sqrt(6. / (n_in + n_out)),
                high=numpy.sqrt(6. / (n_in + n_out)),
                size=(n_in, n_out)
        )
        self.W = theano.shared(value=W_values, name='W', borrow=True)

        b_values = numpy.zeros((n_out,))
        self.b = theano.shared(value=b_values, name='b', borrow=True)

        net = T.dot(input, self.W) + self.b
        self.output = T.nnet.sigmoid(net)

        self.params = [self.W, self.b]
        

In [8]:
class MLNN(object):
    
    def __init__(self, input, n_in, n_hidden):
        self.hiddenLayer = HiddenLayer(
            input=input,
            n_in=n_in,
            n_out=n_hidden
        )
        
        self.logRegressionLayer = LogisticRegression(
            input=self.hiddenLayer.output,
            n_in=n_hidden
        ) 
        
        self.params = self.hiddenLayer.params + self.logRegressionLayer.params
        
    def neg_log_likelihood(self, t):
        return self.logRegressionLayer.neg_log_likelihood(t)
        

In [13]:
x = T.matrix('x')  
y = T.matrix('y')
n_hidden = 20
nn = MLNN(input=x, n_in=n_genes, n_hidden=n_hidden)

In [14]:
cost = nn.neg_log_likelihood(y)

learning_rate = 1e-2
gparams = [T.grad(cost, param) for param in nn.params]
updates = [(param, param-learning_rate*gparam) for param, gparam in zip(nn.params, gparams)]

index = T.lscalar()
train_model = theano.function(
    inputs=[index],
    outputs=cost,
    updates=updates,
    givens={
        x: train_set_x[index * batch_size: (index + 1) * batch_size],
        y: train_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

In [15]:
ep = range(500)
cs = []
for epoch in ep:
    c = 0
    for i in range(n_batches):
        c += train_model(i)
    cs.append(c)